In [1]:
%load_ext autoreload
%autoreload 2

In [43]:
village = "vil55"
layer = "visit"

In [44]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import warnings

import vimure as vm

In [51]:
DATA_INPUT_FOLDER = "../../data/input/india_microfinance/formatted/"  # Folder contains structured CSV files

FILE_PATTERN_EDGES = "%s_edges.csv"
FILE_PATTERN_META = "%s_meta.csv"

ties_layer_mapping={
    "borrowmoney": "money",
    "receivemoney": "money",
    "receiveadvice": "advice",
    "helpdecision": "advice",
    "keroricego": "kerorice",
    "keroricecome": "kerorice",
    "visitgo": "visit",
    "visitcome": "visit",
}
    
df = pd.read_csv(os.path.join(DATA_INPUT_FOLDER, FILE_PATTERN_EDGES % village))
df["layer"] = df["type"].map(ties_layer_mapping)
df.head()

,respondent,i,j,type,weight,layer
0,5500201,5500201,5500302,borrowmoney,1,money
1,5500202,5500202,5503908,borrowmoney,1,money
2,5500301,5500301,5521701,borrowmoney,1,money
3,5500302,5500302,5500402,borrowmoney,1,money
4,5500303,5500303,5501511,borrowmoney,1,money


In [84]:
X = df.rename(columns = {"i": "Ego", "j":"Alter", "respondent":"reporter"}).query("layer == 'money'").reset_index()

X.drop_duplicates().query("Ego != Alter")

,index,reporter,Ego,Alter,type,weight,layer
0,0,5500201,5500201,5500302,borrowmoney,1,money
1,1,5500202,5500202,5503908,borrowmoney,1,money
2,2,5500301,5500301,5521701,borrowmoney,1,money
3,3,5500302,5500302,5500402,borrowmoney,1,money
4,4,5500303,5500303,5501511,borrowmoney,1,money
...,...,...,...,...,...,...,...
1349,1349,5512501,5513301,5512501,receivemoney,1,money
1350,1350,5512502,5512402,5512502,receivemoney,1,money
1351,1351,5518502,5520402,5518502,receivemoney,1,money
1352,1352,5519401,5516601,5519401,receivemoney,1,money


In [105]:
graph = nx.from_pandas_edgelist(
    X, source = "Ego", target = "Alter", edge_attr=True, create_using = nx.MultiDiGraph)

nx.to_pandas_edgelist(graph)

,source,target,reporter,type,index,weight,layer
0,5500201,5500302,5500201,borrowmoney,0,1,money
1,5500201,5500103,5500201,borrowmoney,278,1,money
2,5500302,5500402,5500302,borrowmoney,3,1,money
3,5500302,5502802,5500302,borrowmoney,281,1,money
4,5500302,5502802,5502802,receivemoney,723,1,money
...,...,...,...,...,...,...,...
1349,5507801,5507601,5507601,receivemoney,1051,1,money
1350,5508502,5508407,5508407,receivemoney,1057,1,money
1351,5520502,5515401,5515401,receivemoney,1133,1,money
1352,5520306,5520401,5520401,receivemoney,1174,1,money


In [64]:
rede = vm.io.parse_graph_from_edgelist(X)

/home/gabriela-borges/Git/vimure/Python/src/python/vimure/io.py:213: UserWarning: The set of nodes was not informed, using Ego and Alter columns to infer nodes.
/home/gabriela-borges/Git/vimure/Python/src/python/vimure/io.py:226: UserWarning: The set of reporters was not informed, assuming set(reporters) = set(nodes) and N = M.
2022-06-01 09:23:22,919 - [PID 1510846] - vm.io                     - DEBUG - (1354, 7)
2022-06-01 09:23:22,923 - [PID 1510846] - vm.io                     - DEBUG - (1273, 5)
/home/gabriela-borges/Git/vimure/Python/src/python/vimure/io.py:261: UserWarning: Reporters Mask was not informed (parameter R). Parser will build it from reporter column, assuming a reporter can only report their own ties.
/home/gabriela-borges/Git/vimure/Python/src/python/vimure/io.py:328: UserWarning: Parameter K was None. Defaulting to: 2


In [65]:
rede.X.shape

(1, 482, 482, 482)

In [66]:
rede.R.shape

(1, 482, 482, 482)

In [67]:
vm.model.VimureModel().fit(X=rede.X, R=rede.R)

2022-06-01 09:23:34,542 - [PID 1510846] - vm.model.VimureModel      - DEBUG - Checking user parameters passed to the VimureModel.fit()
/home/gabriela-borges/Git/vimure/Python/src/python/vimure/model.py:165: UserWarning: Parameter K was None. Defaulting to: 2
2022-06-01 09:23:34,602 - [PID 1510846] - vm.model.VimureModel      - DEBUG - Initializing priors
2022-06-01 09:23:34,603 - [PID 1510846] - vm.model.VimureModel      - DEBUG - Setting priors on rho
2022-06-01 09:23:35,668 - [PID 1510846] - vm.model.VimureModel      - DEBUG - Realisation  0 | Iter    1 | ELBO value: -3197.842708465970 | Reached convergence: False
2022-06-01 09:23:40,344 - [PID 1510846] - vm.model.VimureModel      - DEBUG - Realisation  0 | Iter   10 | ELBO value: -1800.190207191989 | Reached convergence: False
2022-06-01 09:23:45,380 - [PID 1510846] - vm.model.VimureModel      - DEBUG - Realisation  0 | Iter   20 | ELBO value: -1714.856510058687 | Reached convergence: False
2022-06-01 09:23:45,382 - [PID 1510846] - 

VimureModel()

In [50]:
["Ego", "Alter"] in diff_columns

False